In [1]:
import os,sys
path_list=[]
roots={"/storage/MARATHONRootfiles/",
       "/v/lustre2/expphy/cache/halla/triton/prod/marathon/"}
for root in roots:
    ddirs = os.listdir(root)
    for ddir in ddirs:
        if( os.path.isdir(root+ddir)):
            for subdir in os.listdir(root+ddir):
                path = root+ddir+"/"+subdir
                path_list.append("{}{}/{}".format(root,ddir,subdir))
    

                

In [54]:

def add_paths(newroot,paths):
    
    ###############
    ddirs = os.listdir(newroot)
    if(os.path.isdir(newroot)):
        paths.append(newroot)
    for ddir in ddirs:
        if( os.path.isdir(root+ddir)):
            for subdir in os.listdir(root+ddir):
                path = root+ddir+"/"+subdir
                paths.append("{}{}/{}".format(root,ddir,subdir))

In [55]:

add_paths("/volatile/halla/triton/Marathon_Rootfiles/pass1/",path_list)

In [56]:
path_list

['/storage/MARATHONRootfiles/pass1/kin11',
 '/storage/MARATHONRootfiles/pass1/kin7',
 '/storage/MARATHONRootfiles/pass1/kin9',
 '/storage/MARATHONRootfiles/pass1/kin2',
 '/storage/MARATHONRootfiles/pass1/kin4',
 '/storage/MARATHONRootfiles/pass1/kin3',
 '/storage/MARATHONRootfiles/pass1/kin15',
 '/storage/MARATHONRootfiles/pass1/kin16',
 '/storage/MARATHONRootfiles/pass1/kin13',
 '/storage/MARATHONRootfiles/pass1/kin1',
 '/storage/MARATHONRootfiles/pass1/kin5',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin13_1st',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin16_1st',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin1p',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin2',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin13_2nd',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin4',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin0',
 '/v/lustre2/expphy/cache/halla/triton/prod/marathon/pas

In [58]:
os.path.exists("/v/lustre2/expphy/cache/halla/triton/prod/marathon/pass2/kin2/tritium_1207.root")

False

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import os,sys
import glob
import numpy as np
import scipy
from scipy import interpolate
from scipy.optimize import curve_fit


import mysql.connector
db_user= 'triton-user'
db_pswd= '3He3Hdata'
db_name= 'triton-work'
db_host= 'halladb'
import SQLFuncs

def SQL_CorList(run):
    comps=["eff","err"]
    allnames=[]
    names=[]
    nums=[]
    effs=[]
    eff={}
    cnx = mysql.connector.connect(user=db_user,host=db_host,database=db_name, password=db_pswd)
    cursor = cnx.cursor(buffered=True)
    query1 = 'select * from MARATHONanalysis where run_number={};'.format(run)
    cursor.execute(query1)
    des =cursor.description
    results=cursor.fetchall()
    cnx.close()  

    if len(results) >= 1:
        for i in range(len(results[0])):
            allnames.append(des[i][0])
        
        for i in range(len(results[0])):
            if comps[0] in des[i][0] :
                if "NE"  in des[i][0] :
                    continue
                names.append(des[i][0])
                nums.append(results[0][i])
                shrt = des[i][0].replace("_eff","")
                lng="{}_err".format(shrt)
                loc = allnames.index(lng)
                effs.append(results[0][loc])
                print(shrt,results[0][i],results[0][loc]  )
                eff[i]={"name":shrt, "eff": results[0][i],"err":results[0][loc] }
         
                
    return names,nums,effs,allnames,eff

def SQL_CorListNE(run):
    comps=["eff","err"]
    allnames=[]
    names=[]
    nums=[]
    effs=[]
    eff={}
    cnx = mysql.connector.connect(user=db_user,host=db_host,database=db_name, password=db_pswd)
    cursor = cnx.cursor(buffered=True)
    query1 = 'select * from MARATHONanalysis where run_number={};'.format(run)
    cursor.execute(query1)
    des =cursor.description
    results=cursor.fetchall()
    cnx.close()  

    if len(results) >= 1:
        for i in range(len(results[0])):
            allnames.append(des[i][0])
        
        for i in range(len(results[0])):
            if comps[0] in des[i][0] :
               # if "NE"  in des[i][0] :
               #     continue
                names.append(des[i][0])
                nums.append(results[0][i])
                shrt = des[i][0].replace("_eff","")
                lng="{}_err".format(shrt)
                loc = allnames.index(lng)
                effs.append(results[0][loc])
                print(shrt,results[0][i],results[0][loc]  )
                if "NE" in shrt:
                    eff[i]={"name":shrt, "eff": results[0][i],"err":results[0][loc] }        
                else:
                    eff[i]={"name":shrt, "eff": results[0][i],"err":results[0][loc] }
         
                
    return names,nums,effs,allnames,eff

def listmult(A,name):
    B=1.0
    index=A.columns
    for i in index:
        if "NE" in A.loc["name"][i]:
            B=B*(1/A.loc[name][i])
        else:
            B=B*A.loc[name][i]
    return B

def inquad(A,name,name1):
    B=0.0
    index=A.columns
    for i in index:
        #if "NE" in A.loc["name"][i]:
        #    B=B+( (1-A.loc[name1][i]) * A.loc[name][i] )**2
        #else :    
        B=B+(A.loc[name][i])**2
    B=np.sqrt(B)
    return B


def get_runcor(run):
    n,nn,nnn,allnames,eff =SQL_CorListNE(run)
    DF1=pd.DataFrame.from_dict(eff)
    eff[len(allnames)]={"name": "Total" , "eff" :listmult(DF1,"eff"), "err" :inquad(DF1,"err","eff")}
    DF1=pd.DataFrame.from_dict(eff)
    DF1=DF1.reindex(['name','eff','err'])
    c=DF1[len(allnames)]
    effnm=c['eff']
    errnm=c['err']
    return effnm,errnm,DF1
    

In [3]:
eff,err,Df=get_runcor(1207)
Df=Df.reindex(['name','eff','err'])
Df

PID_cer 0.997019 0.000543
PID_ps 0.996255 0.001869
PID_sh 0.994934 0.001787
PID_NE 0.999669 0.039157
Tracking 0.985411 0.000997
Trigger 1.0 0.0
livetime 0.98981 0.000715


,9,16,17,20,21,23,26,27
name,PID_cer,PID_ps,PID_sh,PID_NE,Tracking,Trigger,livetime,Total
eff,0.997019,0.996255,0.994934,0.999669,0.985411,1,0.98981,0.964231
err,0.000543,0.001869,0.001787,0.039157,0.000997,0,0.000715,0.0392652


In [84]:
Df


,0
name,Total
eff,1
err,0


In [18]:
import pandas as pd
DF=pd.DataFrame.from_dict(eff,or)

In [19]:
DF=DF.reindex(['name','eff','err'])

In [20]:
DF

,9,16,17,21,23,26
name,PID_cer,PID_ps,PID_sh,Tracking,Trigger,livetime
eff,0.996759,0.994516,0.991634,0.986878,0.999949,0.949222
err,0.000163,0.000631,0.000632,0.000252,0,0.000442


In [105]:
eff[27]={"name": "Total" , "eff" :listmult(DF,"eff"), "err" :inquad(DF,"err","eff")}

In [106]:
DF1=pd.DataFrame.from_dict(eff)

In [107]:
DF1.loc["name"][26]

'livetime'

In [108]:
DF1=DF1.reindex(['name','eff','err'])
DF1

,9,16,17,20,21,23,26,27
name,PID_cer,PID_ps,PID_sh,PID_NE,Tracking,Trigger,livetime,Total
eff,0.99714,0.993948,0.991138,0.999616,0.987001,0.999907,0.95347,0.920794
err,0.000191,0.000829,0.000812,0.013924,0.00025,0,0.000317,0.00102511


In [117]:

c=DF1[27]
eff=c['eff']
err=c['err']


0.9207939612133124

In [250]:
0.999589/0.96269

1.0383290571211916

In [251]:
(1/0.999589)*0.96269

0.9630858282754213

In [252]:
np.sqrt(0.00319572**2 + 0.045313**2 )

0.04542555002769256

In [93]:
n,nn,nnn,allnames,eff =SQL_CorListNE(1222)
DF1=pd.DataFrame.from_dict(eff)
DF1=DF1.reindex(['name','eff','err'])

In [92]:
nnn

[0.469931, 0.000372, 0.000352, 0.0, 0.00025, 0.0, 0.000317]

In [25]:
DF1=pd.DataFrame.from_dict(eff, orient='index')

In [26]:

DF1

,name,eff,err
9,PID_cer,0.997140,0.000191
16,PID_ps,0.993948,0.000829
17,PID_sh,0.991138,0.000812
20,PID_NE,0.999616,0.013924
21,Tracking,0.987001,0.000250
23,Trigger,0.999907,0.000000
26,livetime,0.953470,0.000317


In [47]:
eff[27]={"name": "Total" , "eff" :listmult(DF1,"eff"), "err" :inquad(DF1,"err","eff")}

In [48]:
DF1=pd.DataFrame.from_dict(eff)

In [49]:
DF1

,9,16,17,20,21,23,26,27
eff,0.99714,0.993948,0.991138,0.999616,0.987001,0.999907,0.95347,0.924709
err,0.000191,0.000829,0.000812,0.013924,0.00025,0,0.000317,0.013973
name,PID_cer,PID_ps,PID_sh,PID_NE,Tracking,Trigger,livetime,Total


In [78]:
list(eff)

['name', 'eff', 'err']

In [102]:
A =eff.index()

AttributeError: 'dict' object has no attribute 'index'

In [101]:
A

dict_values([{'name': 'PID_cer', 'eff': 0.99714, 'err': 0.000191}, {'name': 'PID_ps', 'eff': 0.993948, 'err': 0.000829}, {'name': 'PID_sh', 'eff': 0.991138, 'err': 0.000812}, {'name': 'PID_NE', 'eff': 0.999616, 'err': 0.013924}, {'name': 'Tracking', 'eff': 0.987001, 'err': 0.00025}, {'name': 'Trigger', 'eff': 0.999907, 'err': 0.0}, {'name': 'livetime', 'eff': 0.95347, 'err': 0.000317}, {'name': 'Total', 'eff': 0.9247087455910981, 'err': 0.013972976676454668}])

In [92]:
print(A)

<built-in method items of dict object at 0x7f23222784c8>


In [93]:
len(A)

TypeError: object of type 'builtin_function_or_method' has no len()